In [1]:
from datetime import datetime
import pandas as pd
import re
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import nltk
import gensim

/home/yesemsanthoshkumar/anaconda3/lib/python3.5/site-packages/nltk/decorators.py:59: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  regargs, varargs, varkwargs, defaults = inspect.getargspec(func)
/home/yesemsanthoshkumar/anaconda3/lib/python3.5/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
rndm=2486

In [3]:
df = pd.read_csv("data.csv")

In [4]:
df.shape

(108129, 14)

In [5]:
text_cols = ['desc']

In [6]:
wrd_net_lemmatizer = nltk.stem.WordNetLemmatizer()
tokenizer = nltk.tokenize.RegexpTokenizer("[\w]+")
stops_en = nltk.corpus.stopwords.words('english')

In [7]:
def preprocess_keywords(corp):
    corp = corp.split('-')
    corp = " ".join(corp)
    tokens = tokenizer.tokenize(corp)
    tokens = [
        t.lower()
        for t in tokens
    ]
    tokens = [
        t
        for t in tokens
        if t not in stops_en
    ]
    tokens = [
        wrd_net_lemmatizer.lemmatize(t)
        for t in tokens
    ]
    tokens = [
        t
        for t in tokens
        if t.isalpha() and len(t)>=3
    ]
    return tokens

In [8]:
def tfidf_preprocess(content):
    content = re.sub('\$?(?P<digits>\d+)(d|st|th|rd)?', '\g<digits>', content)
    content = re.sub('\w(?=\1){3, }', '\1', content).split()
    content = [
        wrd_net_lemmatizer.lemmatize(ct)
        for ct in content
    ]
    content = [
        x
        for x in content
        if len(x) > 2
    ]
    return " ".join(content)

In [9]:
corpus = df['keywords'].apply(preprocess_keywords)

In [10]:
corp_dict = gensim.corpora.dictionary.Dictionary(corpus)

In [11]:
dt_matrix = [
    corp_dict.doc2bow(x)
    for x in corpus
]

In [12]:
lda_model = gensim.models.ldamodel.LdaModel(
    dt_matrix,
    num_topics=8,
    id2word=corp_dict,
    passes=2,
    random_state=rndm
)

In [13]:
pyLDAvis.gensim.prepare(
    lda_model,
    dt_matrix,
    corp_dict
)

/home/yesemsanthoshkumar/anaconda3/lib/python3.5/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      15.617007        1       1 -0.185812  0.003323
2      12.629336        1       2  0.126647  0.320888
4      12.232367        1       3  0.278772 -0.133822
3      12.185575        1       4  0.009930  0.057096
5      12.056899        1       5  0.020594 -0.151424
1      11.851464        1       6 -0.110519 -0.062604
0      11.750074        1       7 -0.152376  0.040309
6      11.677280        1       8  0.012763 -0.073766, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
33675  Default  2098.000000          film  2098.000000  30.0000  30.0000
322    Default  1990.000000          game  1990.000000  29.0000  29.0000
43268  Default  1879.000000          help  1879.000000  28.0000  28.0000
35694  Default  2126.000000          book  2126.000000  27.0000  27.0000
21895  Default  1721.000000         music  1721.000000  26.0000  26.0000
14421  Default  1201.000000           app  1201.000000  25.0000  25.0000
4634   Default  1132.000000          food  1132.000000  24.0000  24.0000
28583  Default  2329.000000         album  2329.000000  23.0000  23.0000
36494  Default  1133.000000         short  1133.000000  22.0000  22.0000
2673   Default  2740.000000           art  2740.000000  21.0000  21.0000
20214  Default  1002.000000         debut  1002.000000  20.0000  20.0000
13385  Default  1030.000000          card  1030.000000  19.0000  19.0000
25194  Default  2743.000000           new  2743.000000  18.0000  18.0000
49309  Default   987.000000         video   987.000000  17.0000  17.0000
25872  Default   995.000000          love   995.000000  16.0000  16.0000
19403  Default  1457.000000         first  1457.000000  15.0000  15.0000
15288  Default   901.000000        series   901.000000  14.0000  14.0000
1859   Default   875.000000         novel   875.000000  13.0000  13.0000
19648  Default   873.000000      clothing   873.000000  12.0000  12.0000
10712  Default   860.000000          make   860.000000  11.0000  11.0000
49907  Default   900.000000        studio   900.000000  10.0000  10.0000
5196   Default   757.000000          tour   757.000000   9.0000   9.0000
49825  Default   701.000000          free   701.000000   8.0000   8.0000
27003  Default   692.000000        design   692.000000   7.0000   7.0000
29289  Default   793.000000          need   793.000000   6.0000   6.0000
12453  Default   682.000000         comic   682.000000   5.0000   5.0000
39018  Default   703.000000        mobile   703.000000   4.0000   4.0000
23547  Default   902.000000           one   902.000000   3.0000   3.0000
37724  Default   640.000000         truck   640.000000   2.0000   2.0000
23122  Default  1491.000000          life  1491.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
42078   Topic8   305.921448        action   306.715271   2.1449  -5.0793
55327   Topic8   274.452148           box   275.245850   2.1446  -5.1879
18646   Topic8   268.396393         color   269.190491   2.1446  -5.2102
3139    Topic8   265.328094        change   266.122009   2.1445  -5.2217
40167   Topic8   263.835846         radio   264.629425   2.1445  -5.2273
46670   Topic8   252.508987         sweet   253.303085   2.1444  -5.2712
6690    Topic8   251.965775      bracelet   252.762009   2.1444  -5.2734
9794    Topic8   244.762466         night   245.556122   2.1443  -5.3024
17029   Topic8   241.354584        gaming   242.148209   2.1442  -5.3164
31731   Topic8   235.949875         water   236.743622   2.1442  -5.3390
48764   Topic8   232.796906          wolf   233.591309   2.1441  -5.3525
9467    Topic8   223.505219        medium   224.300293   2.1440  -5.3932
19461   Topic8   222.279037      learning   223.073532   2.1440  -5.3987
32297   Topic8   218.395187         sport   219.188858   2.